In [ ]:
#assume: tokenization, lemmatization, vectorization are in place

#assume: train, test set available
# train = df['reviews.rating']
# test = df['reviews.rating']

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline

# Set the variable for the algo TO-DO: check what makes sense, naive or model variable
# naive=MultinomialNB()

# Creating model based on Multinomial Naive Bayes
model = make_pipeline (TfidVectorizer(), MultinomialNB)

# Training the model with the train data
model.fit(traindataset, train['reviews.rating'])




In [ ]:
# Creating labels for the test data
labels = model.predict(test.data)

In [ ]:
# Predict for the Test Dataset
test_transform= []
for row in range(0,len(test.index)):
    test_transform.append(' '.join(str(x) for x in test.iloc[row,2:27]))
test_dataset = tfidfvector.transform(test_transform)
predictions = naive.predict(test_dataset)

In [ ]:
# Evaluation
matrix=confusion_matrix(test['reviews.rating'],predictions)
print(matrix)
score=accuracy_score(test['reviews.rating'],predictions)
print(score)
report=classification_report(test['reviews.rating'],predictions)
print(report)